In [ ]:
# https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial15/Vision_Transformer.html Helpful tutorials
# curl -fsSL https://ollama.ai/install.sh | sh
# ollama run llama3.2:1b

# https://dspy.ai/tutorials/rag/

In [21]:
import dspy
lm = dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [22]:
lm("Say this is a test!", temperature=0.7)  # => ['This is a test!']
lm(messages=[{"role": "user", "content": "Say this is a test!"}])  # => ['This is a test!']

["It looks like you're getting into the spirit of things. That's completely fine. I'm here to help and provide information, so feel free to ask me anything, no matter what it is. What's on your mind?"]

In [23]:
from datasets import load_dataset
dataset = "openlifescienceai/medmcqa"

#Create the Dataset to create prompts.
data = load_dataset(dataset)

train_sample = data["train"].select(range(1000))


In [24]:

train_sample = train_sample.remove_columns(['id','opa','opb','opc','cop','opd','choice_type','subject_name','topic_name'])

display(train_sample)



Dataset({
    features: ['question', 'exp'],
    num_rows: 1000
})

In [25]:
display(train_sample[0])

{'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950'}

In [26]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print(response.title)
print(response.headings)
print(response.entities)

Apple Inc. announces latest iPhone 14
['# Apple Announces Latest iPhone 14', '# Features of the New iPhone 14']
[{}, {}]


In [27]:
math = dspy.ChainOfThought("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals two?")

Prediction(
    reasoning='To calculate this, we need to consider all possible outcomes of rolling two dice and count how many of them result in a sum of 2.\n\nThere are 6x6 = 36 possible outcomes when rolling two dice (each die has 6 faces).\n\nThe combinations that result in a sum of 2 are: (1,1)\n\nSo, the probability is 1/36.',
    answer=0.02777777777777778
)

In [28]:

# Define the DSPy module for classification. It will use the hint at training time, if available.
signature = dspy.Signature("text -> label").with_updated_fields('label', type_=Literal[tuple(CLASSES)])
classify = dspy.ChainOfThoughtWithHint(signature)

# Optimize via BootstrapFinetune.
optimizer = dspy.BootstrapFinetune(metric=(lambda x, y, trace=None: x.label == y.label), num_threads=24)
optimized = optimizer.compile(classify, trainset=trainset)

optimized(text="What does a pending cash withdrawal mean?")

NameError: name 'Literal' is not defined

In [29]:


qa = dspy.Predict('question: str -> response: str')
response = qa(question="what are high memory and low memory on linux?")

print(response.response)


High Memory: 8GB or more
Low Memory: Less than 4GB


In [31]:
cot = dspy.ChainOfThought('question -> response')
cot(question="should curly braces appear on their own line?")

Prediction(
    reasoning='{reasoning}',
    response='{response}'
)

In [43]:
train_sample = train_sample.rename_column('exp','response')

In [44]:
data = [dspy.Example(**d).with_inputs('question') for d in train_sample]

In [45]:
example = data[2]
example

Example({'question': 'All of the following are surgical options for morbid obesity except -', 'response': "Ans. is 'd' i.e., Roux en Y Duodenal Bypass Bariatric surgical procedures include:a. Vertical banded gastroplastyb. Adjustable gastric bandingc. Roux-en Y gastric bypass (Not - Roux-en Y Duodenal Bypass)d. Biliopancreatic diversione. Duodenal switcho The surgical treatment of morbid obesity is known as bariatric surgery.o Morbid obesity is defined as body mass index of 35 kg/m2 or more with obesity related comorbidity, or BMI of 40 kg/m2 or greater without comorbidity.o Bariatric operations produce weight loss as a result of 2 factors. One is restriction of oralintake. The other is malabsorbtion of ingested food.o Gastric restrictive procedures include Vertical banded gastroplasty & Adjustable gastric bandingo Malabsorbtive procedures include Biliopancreatic diversion, and Duodenal switcho Roux-en Y gastric bypass has features of both restriction and malabsorptionBariatric Operati

In [46]:


import random

random.Random(0).shuffle(data)
trainset, devset, testset = data[:200], data[200:500], data[500:1000]

len(trainset), len(devset), len(testset)


(200, 300, 500)

In [47]:
from dspy.evaluate import SemanticF1

# Instantiate the metric.
metric = SemanticF1(decompositional=True)

# Produce a prediction from our `cot` module, using the `example` above as input.
pred = cot(**example.inputs())

# Compute the metric score for the prediction.
score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

Question: 	 All of the following are surgical options for morbid obesity except -

Gold Response: 	 Ans. is 'd' i.e., Roux en Y Duodenal Bypass Bariatric surgical procedures include:a. Vertical banded gastroplastyb. Adjustable gastric bandingc. Roux-en Y gastric bypass (Not - Roux-en Y Duodenal Bypass)d. Biliopancreatic diversione. Duodenal switcho The surgical treatment of morbid obesity is known as bariatric surgery.o Morbid obesity is defined as body mass index of 35 kg/m2 or more with obesity related comorbidity, or BMI of 40 kg/m2 or greater without comorbidity.o Bariatric operations produce weight loss as a result of 2 factors. One is restriction of oralintake. The other is malabsorbtion of ingested food.o Gastric restrictive procedures include Vertical banded gastroplasty & Adjustable gastric bandingo Malabsorbtive procedures include Biliopancreatic diversion, and Duodenal switcho Roux-en Y gastric bypass has features of both restriction and malabsorptionBariatric Operations: Me

In [ ]:
#https://github.com/stanfordnlp/dspy/blob/main/dspy/evaluate/auto_evaluation.py

In [48]:
# Define an evaluator that we can re-use.
evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=10,
                         display_progress=True, display_table=2)

# Evaluate the Chain-of-Thought program.
evaluate(cot)

Average Metric: 124.81 / 176 (70.9%):  59%|████████████████████████████████████████████████████████████████████████████▊                                                      | 176/300 [04:51<04:01,  1.95s/it]

2025/04/17 08:43:58 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: Expected dict_keys(['reasoning', 'ground_truth_key_ideas', 'system_response_key_ideas', 'discussion', 'recall', 'precision']) but got dict_keys(['reasoning', 'ground_truth_key_ideas', 'system_response_key_ideas'])


Average Metric: 129.86 / 183 (71.0%):  61%|███████████████████████████████████████████████████████████████████████████████▉                                                   | 183/300 [05:03<02:31,  1.29s/it]

2025/04/17 08:44:10 ERROR dspy.utils.parallelizer: Error for Example({'question': 'Carotid artery stenosis screening invivo choice is -', 'response': "Ans. is 'd' i.e., Doppler o Color doppler ultrasonography is the screening method of choice (not simple USG : option a is incorrect),o Carotid angiography is the investigation of choice for diagnosis,o MRI is the investigation of choice for demyelinating disorder, e.g. multiple sclerosis.Investigations in nervous systemo Investigation of choice for white matter disease - MRI (CT is second choice),o Investigation of choice of acoustic neuroma - GD enhanced MRIo Best investigation for all brain tumors - Contrast enhanced MRIo Investigation of choice for meningeal carcinomatosis - Contrast (gadolinium) enhanced MRI.o Investigation of choice for posterior fossa tumor - MRIo Investigation of choice for para meningeal rhabdomyosarcoma - MRIo Investigation of choice for nasopharyngeal angiofibroma - Contrast enhanced CT.o Investigation of choic

Average Metric: 215.60 / 299 (72.1%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [07:56<00:00,  1.59s/it]

2025/04/17 08:47:02 INFO dspy.evaluate.evaluate: Average Metric: 215.60204357717578 / 300 (71.9%)


,question,example_response,reasoning,pred_response,SemanticF1,response
0,Tigroid white matter on MRI is seen in?,The tigroid pattern/ leopard skin sign occurs on MRI head is seen ...,Tigroid white matter on MRI is typically associated with brain inj...,"This pattern can also be seen in cases of traumatic brain injury, ...",✔️ [0.824],NaN
1,Fries test is used in the diagnosis of:,Granuloma venereum,The Fries test is a diagnostic tool used to assess the presence of...,"In general, the Fries test is used to help diagnose conditions tha...",✔️ [0.686],NaN


71.87

In [ ]:
# https://dspy.ai/tutorials/rag/ Next steps to enhance this with RAG